In [2]:
import xlrd
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
import xlwt
import operator
import copy
from functools import reduce
import sklearn.cluster as sc
import re
import bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook() #导入这个命令之后bokeh的绘画内容都在notebook上显示

# 不用科学计数
pd.set_option('display.float_format',lambda x : '%.2f' % x)

Loading BokehJS ...

# 数据预处理

In [8]:
hair_dryer=pd.read_csv('./data_c/hair_dryer.tsv', sep='\t')# 吹风机

microwave=pd.read_csv('./data_c/microwave.tsv', sep='\t')# 微波炉
pacifier=pd.read_csv('./data_c/pacifier.tsv', sep='\t')# 奶嘴

In [15]:
def number_of_data(data_origin):
    '''
    计算并存储各类数据共多少类，返回字典，字典长度即数目
    '''
    country={}
    for country_i in range(data_origin.shape[0]):# ii 是从0开始的
        #FIPS_Combined=data_origin.iloc[county_i][5]
        country_id=data_origin.iloc[country_i][0]
        if country_id not in country.keys():
            country[country_id]=country_id
            
    return country

In [16]:
number_of_data(hair_dryer)

{'US': 'US'}

In [17]:
number_of_data(microwave)

{'US': 'US', 'us': 'us'}

In [18]:
number_of_data(pacifier)

{'US': 'US', 'us': 'us'}

第一个数据项只有US，考虑剔除

In [21]:
hair_dryer_notVerify=hair_dryer[hair_dryer['verified_purchase']=='N']
hair_dryer_notVerify

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
4,US,107098,R35BHQJHXXJD59,B003V264WW,732252283,remington ac2015 t|studio salon collection pea...,Beauty,4,0,0,N,N,I think's great. The cord length is perfect,I just got this last week. I think's great. Th...,8/31/2015
6,US,39431051,R21NN9ONVZITI0,B000FS1W4U,235105995,"revlon essentials 1875w fast dry hair dryer, r...",Beauty,1,0,0,N,N,Gets extremely hot - I have burned my hand on ...,Gets extremely hot - I have burned my hand on ...,8/31/2015
10,US,181650,R2NEOR5Y0P7U8V,B003FBG88E,195677102,conair pro styler ionic conditioning hair dryer,Beauty,5,0,0,N,N,Awesome dryer,Got tho on super sale. Love it! Cuts my drying...,8/30/2015
28,US,32500930,REOAJPZWKR6K9,B00VRN7SB8,253917972,remington silk ceramic professional hair dryer,Beauty,5,0,0,Y,N,Great Hair Dryer,This is a really nice hair dryer from Remingto...,8/28/2015
44,US,51915611,RP1Z1HVOLQD34,B00006IV22,357308868,conair 1875 watt dual voltage folding handle h...,Beauty,5,0,0,N,N,Excellent value for travel dryer on a budget,Compact yet powerful. Easy to pack and travel ...,8/27/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11463,US,50034058,R1BLPTSZRE4134,B000065DJY,919751065,revlon 1875w volumizing hair dryer,Beauty,5,59,61,N,N,Excellent! Perfect for my thick hair.,"I have thick shoulder length hair, that usuall...",12/17/2002
11464,US,49993301,R33QQ6N5UIPSY1,B000065DJY,919751065,revlon 1875w volumizing hair dryer,Beauty,4,4,4,N,N,Close to Perfect,"Overall, I am extremely pleased with this hair...",11/8/2002
11465,US,35599734,R2O50YNP83CG34,B000065DJY,919751065,revlon 1875w volumizing hair dryer,Beauty,5,14,18,N,N,great hair!,"I just purchased this dryer, and I really like...",8/21/2002
11466,US,43977044,R2JQPUYU65C4QD,B00005O0MZ,694290590,conair corp pers care 146np conair ionic condi...,Beauty,1,26,32,N,N,Disappointment with dryer,I purchased it because it was supposed to be q...,8/13/2002


verified_purchase (string)：“Y”表示，亚马逊证实撰写评论的人在亚马逊购买了该产品，并没有以较低的折扣收到该产品。

发现确实有未核实的购买数据，考虑删除。

得到经过核实的数据认为是有效的数据

In [25]:
hair_dryer_Verify=hair_dryer[hair_dryer['verified_purchase']=='Y']# 得到有效评论
hair_dryer_Verify.drop(['marketplace'],axis=1,inplace=True)
hair_dryer_Verify

E:\anaconda\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,34678741,R9T1FE2ZX2X04,B003V264WW,732252283,remington ac2015 t|studio salon collection pea...,Beauty,5,0,0,N,Y,Works great,Works great!,8/31/2015
1,11599505,RE36JAD5V53PO,B0009XH6V4,670161917,andis micro turbo hair dryer,Beauty,4,0,0,N,Y,I love travel blow dryers because they are eas...,"This dries my hair faster that bigger, more po...",8/31/2015
2,2282190,RIDHM8B7SCCV3,B0007NZPY6,16483457,conair pro hair dryer,Beauty,5,0,1,N,Y,Five Stars,Love this dryer!,8/31/2015
3,43669858,R14QGWPCHU9LSE,B00BB8ZIW0,253917972,remington silk ceramic professional hair dryer,Beauty,5,0,0,N,Y,Five Stars,styling hair in style,8/31/2015
5,51995766,R230LCPQDOFJJZ,B000065DJY,919751065,revlon 1875w volumizing hair dryer,Beauty,5,0,0,N,Y,Five Stars,Excellent dryer.,8/31/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11436,38641465,R37KYGDKHQ87NK,B0009XH6TG,47684938,"andis 1875-watt fold-n-go ionic hair dryer , s...",Beauty,4,263,270,N,Y,"Great product and the only ceramic, ionic blow...","This is the only ceramic, ionic blow dryer wit...",9/15/2005
11442,51785663,R3NZ6I1EKJ9FQL,B0002G214U,685652978,conair soft bonnet hair dryer 1 ea,Beauty,1,165,168,N,Y,Very Poor Product,I wish I could have read reviews on this produ...,5/25/2005
11446,50473837,R12APPEFY714W7,B00005O0MZ,694290590,conair corp pers care 146np conair ionic condi...,Beauty,2,5,6,N,Y,a Bit Disappointed,The on/off switch on this dryer needs improvem...,8/15/2004
11467,37733836,R3GO6L5PWBS0IW,B00005O0MZ,694290590,conair corp pers care 146np conair ionic condi...,Beauty,5,4,4,N,Y,The best dryer!!!,"I love this dryer not only is it QUIET, it rea...",7/13/2002


In [31]:
hair_dryer_Verify['helpful_votes'].max()

499

In [32]:
hair_dryer_Verify[hair_dryer_Verify['helpful_votes']==499]

,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
10575,32455129,R2X4H7F2PTT7R8,B00132ZG3U,758099411,conair 1875 watt tourmaline ceramic hair dryer,Beauty,5,499,575,N,Y,A review for guys,I didn't know what the attachments were for so...,3/10/2010


# 星级，评论和帮助等级（helpfulness ratings）之间的关系

In [39]:
def number_of_data_review(data_origin):
    '''
    计算并存储各类数据共多少类，返回字典，字典长度即数目
    '''
    country={}
    for country_i in range(data_origin.shape[0]):# ii 是从0开始的
        #FIPS_Combined=data_origin.iloc[county_i][5]
        country_id=data_origin.iloc[country_i][11]
        if country_id not in country.keys():
            country[country_id]=1
        else :#如果已经在里面了，计算出现频率
            country[country_id]=country[country_id]+1
    return country

### 计算商品有多少种

In [73]:
def number_of_data_product(data_origin):
    '''
    计算并存储各类数据共多少类，返回字典，字典长度即数目
    '''
    country={}
    for country_i in range(data_origin.shape[0]):# ii 是从0开始的
        #FIPS_Combined=data_origin.iloc[county_i][5]
        country_id=data_origin.iloc[country_i][3]
        if country_id not in country.keys():
            country[country_id]=country_id
    return country

In [75]:
type(hair_dryer_Verify.iloc[0,3])
len(number_of_data_product(hair_dryer_Verify))

364

共364种商品

### 删除标点符号，全部换为小写

In [68]:
hair_dryer_Verify.iloc[:,11].str.lower().str.replace(r'[^\w\s]+', '')

0                                              works great
1        i love travel blow dryers because they are eas...
2                                               five stars
3                                               five stars
5                                               five stars
                               ...                        
11436    great product and the only ceramic ionic blow ...
11442                                    very poor product
11446                                   a bit disappointed
11467                                       the best dryer
11469                             some pluses some minuses
Name: review_headline, Length: 9811, dtype: object

In [40]:

review=number_of_data_review(hair_dryer_Verify)
review

{'Works great': 18,
 'I love travel blow dryers because they are easy to lift ...': 1,
 'Five Stars': 1221,
 'Everything okay but.....!!': 1,
 'Nice hairdryer that works very well.': 1,
 'Works great, I use it 5 days a week': 1,
 'The styling tip does not stay on,': 1,
 'Blows like a jet engine!': 1,
 'Great drier...........meh': 1,
 'A good replacement': 1,
 'Girlfriend loves it': 3,
 'Nice, light and powerful...': 1,
 'Great Blow Dryer': 6,
 'HOT AIR!': 1,
 'Works just like the ones in the hotels.': 1,
 'I am obsessed': 1,
 'Works great. Just wish it had a medium setting': 1,
 'No complaints!': 2,
 'Two Stars': 60,
 'Great blowdryer. it gets the job done': 1,
 'Well worth the money!': 1,
 'Buy this': 1,
 'Great little dryer!': 5,
 'Perfect': 17,
 'Great blow dryer': 10,
 'Great dryer for travel': 1,
 'This Conair Is Full of Air': 1,
 'was easy to mount on the wall': 1,
 'Petite, but Powerful Performance': 1,
 'One Star': 79,
 'but it worked great. Dried my hair in about 15 minutes': 

In [41]:
import torchtext

In [52]:
review_df=pd.Series(review)
review_df=review_df.sort_values(ascending=False)
review_df

Five Stars                             1221
Four Stars                              269
Three Stars                             122
One Star                                 79
Two Stars                                60
                                       ... 
It operates as advertised.                1
My Hair Dryer of Choice                   1
Best Hair Dryer My Wife ever Owned.       1
Finally found a great hairdryer!          1
Portable Dryer                            1
Length: 6428, dtype: int64

In [53]:
writer = pd.ExcelWriter('review_df.xlsx')
review_df.to_excel(writer, 'Sheet1')
writer.save()

In [55]:
microwave_Verify=microwave[microwave['verified_purchase']=='Y']
microwave_Verify.drop(['marketplace'],axis=1,inplace=True)
microwave_Verify

E:\anaconda\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,21879631,RY52KZABZK8QF,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,Major Appliances,1,0,0,N,Y,Do not recommend it. go use your money for som...,3 of the buttons stopped working after a month...,8/31/2015
1,14964566,R3GCOEV4HYZG2I,B0055UBB4O,423421857,danby 0.7 cu.ft. countertop microwave,Major Appliances,5,0,0,N,Y,Looks as good as the picture,Didn't use it long so I can't attest to that b...,8/31/2015
2,13230389,R1V2OPPNL0QGCE,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,Major Appliances,4,0,0,N,Y,Four Stars,"Very nice microwave, great price",8/31/2015
3,43655888,R9Q0QDTLKV567,B004ZU09QQ,423421857,danby 0.7 cu.ft. countertop microwave,Major Appliances,3,0,1,N,Y,quiet,"Quiet, but does not seem like 1000 watt power.",8/31/2015
4,117794,R3DL7HYC3QTWNI,B005GSZB7I,827502283,"whirlpool stainless look countertop microwave,...",Major Appliances,4,0,0,N,Y,Four Stars,It's bigger than I thought.,8/31/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,30558664,R124A9RCE0C1EI,B00012ORT2,305608994,sharp 950-watt 1-2/5-cubic-foot over-the-range...,Major Appliances,5,93,98,N,Y,Pleased I did not follow advise of other revie...,I bought this on Amazon to replace a Sharp mic...,1/20/2005
1605,36083583,R19SLY8AEJ49RX,B00009V3WS,459626087,sharp 1.1-cubic-foot 850-watt over-the-range c...,Major Appliances,3,56,61,N,Y,It was a bit of a disappointment,When we were remodeling our kitchen we decided...,12/10/2004
1606,20973683,R1DKTZQ1KA9IFB,B00009V3X6,459626087,sharp 1.1-cubic-foot 850-watt over-the-range c...,Major Appliances,1,16,19,N,Y,Bargain??? Mind the interior!,"One week after installing this over-the-stove,...",11/20/2004
1612,36386173,RXOAWGI9Z98QY,B00009V3WZ,379992322,sharp 1-1/2-cubic-foot 1000-watt over-the-rang...,Major Appliances,4,19,19,N,Y,"Great microwave, little tough to mount",Its a pretty big and heavy unit so having some...,6/25/2004


In [59]:
microwave_review=number_of_data_review(microwave_Verify)
microwave_review=pd.Series(microwave_review)
microwave_review=microwave_review.sort_values(ascending=False)
microwave_review


Five Stars                                                       145
Four Stars                                                        35
One Star                                                          13
Three Stars                                                       10
Great microwave                                                    5
                                                                ... 
Best small microwave!                                              1
Good and bad                                                       1
super in a small size                                              1
received dented unit                                               1
Do not recommend it. go use your money for something more ...      1
Length: 847, dtype: int64

### 分析帮助程度和产品评级之间的关系
需要对帮助程度做一个指标分析，如果total_votes==0则帮助程度为0

if total_votes not == 0：（helpful_votes/total_votes ）\*f(total_votes)

一个关于total_votes的函数，可以取logistic()平移后的函数（向右平移total_votes(max)/2）

#### 带偏置sigmoid

In [83]:
import math
import matplotlib.pyplot as plt
 
def sigmoid(x):
    a = []
    for item in x:
        a.append(1.0/(1.0 + math.exp(-item)))
    return a

In [ ]:
max_votes=data['total_votes'].max()
colum_of_totalVote=8
for i in range (len(data)):
    total_votes=data.iloc[i,colum_of_totalVote]
    helpful_votes=data.iloc[i,colum_of_totalVote-1]
    if total_votes==0:
        help_degree=0
    else:
        help_degree=(helpful_votes/total_votes)*

In [79]:
microwave_Verify

,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,21879631,RY52KZABZK8QF,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,Major Appliances,1,0,0,N,Y,Do not recommend it. go use your money for som...,3 of the buttons stopped working after a month...,8/31/2015
1,14964566,R3GCOEV4HYZG2I,B0055UBB4O,423421857,danby 0.7 cu.ft. countertop microwave,Major Appliances,5,0,0,N,Y,Looks as good as the picture,Didn't use it long so I can't attest to that b...,8/31/2015
2,13230389,R1V2OPPNL0QGCE,B0052G14E8,423421857,danby 0.7 cu.ft. countertop microwave,Major Appliances,4,0,0,N,Y,Four Stars,"Very nice microwave, great price",8/31/2015
3,43655888,R9Q0QDTLKV567,B004ZU09QQ,423421857,danby 0.7 cu.ft. countertop microwave,Major Appliances,3,0,1,N,Y,quiet,"Quiet, but does not seem like 1000 watt power.",8/31/2015
4,117794,R3DL7HYC3QTWNI,B005GSZB7I,827502283,"whirlpool stainless look countertop microwave,...",Major Appliances,4,0,0,N,Y,Four Stars,It's bigger than I thought.,8/31/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,30558664,R124A9RCE0C1EI,B00012ORT2,305608994,sharp 950-watt 1-2/5-cubic-foot over-the-range...,Major Appliances,5,93,98,N,Y,Pleased I did not follow advise of other revie...,I bought this on Amazon to replace a Sharp mic...,1/20/2005
1605,36083583,R19SLY8AEJ49RX,B00009V3WS,459626087,sharp 1.1-cubic-foot 850-watt over-the-range c...,Major Appliances,3,56,61,N,Y,It was a bit of a disappointment,When we were remodeling our kitchen we decided...,12/10/2004
1606,20973683,R1DKTZQ1KA9IFB,B00009V3X6,459626087,sharp 1.1-cubic-foot 850-watt over-the-range c...,Major Appliances,1,16,19,N,Y,Bargain??? Mind the interior!,"One week after installing this over-the-stove,...",11/20/2004
1612,36386173,RXOAWGI9Z98QY,B00009V3WZ,379992322,sharp 1-1/2-cubic-foot 1000-watt over-the-rang...,Major Appliances,4,19,19,N,Y,"Great microwave, little tough to mount",Its a pretty big and heavy unit so having some...,6/25/2004


In [80]:
microwave_Verify['degree']=microwave_Verify['helpful_votes']/microwave_Verify['total_votes']

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
microwave_Verify['degree']=microwave_Verify['degree'].fillna(0)
x=list(microwave_Verify['total_votes'])

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [94]:
sigmoid_x=sigmoid(x)

In [95]:
microwave_Verify['degree']=microwave_Verify['degree']*sigmoid_x

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
microwave_Verify['degree']

0      0.00
1      0.00
2      0.00
3      0.00
4      0.00
       ... 
1604   0.95
1605   0.92
1606   0.84
1612   1.00
1613   1.00
Name: degree, Length: 1089, dtype: float64

In [97]:
sigmoid_x

[0.5,
 0.5,
 0.5,
 0.7310585786300049,
 0.5,
 0.5,
 0.5,
 0.5,
 0.7310585786300049,
 0.5,
 0.5,
 0.7310585786300049,
 0.7310585786300049,
 0.7310585786300049,
 0.9990889488055994,
 0.8807970779778823,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.8807970779778823,
 0.8807970779778823,
 0.5,
 0.5,
 0.5,
 0.5,
 0.8807970779778823,
 1.0,
 0.5,
 0.5,
 0.5,
 0.5,
 0.7310585786300049,
 0.9525741268224334,
 0.7310585786300049,
 0.8807970779778823,
 0.7310585786300049,
 0.7310585786300049,
 0.5,
 0.9998766054240137,
 0.7310585786300049,
 0.9820137900379085,
 0.5,
 0.7310585786300049,
 0.9525741268224334,
 0.5,
 0.5,
 0.5,
 0.9820137900379085,
 0.9525741268224334,
 0.7310585786300049,
 0.7310585786300049,
 0.5,
 0.5,
 0.5,
 0.5,
 0.8807970779778823,
 0.8807970779778823,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.7310585786300049,
 0.5,
 0.9999999997210531,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.8807970779778823,
 0.5,
 0.7310585786300049,
 0.5,
 0.5,
 0.5,
 0.9525741268224334,
 0.5,
 0.8807970779778823,
 0.9999991684

In [96]:
microwave_Verify['degree']

0      0.00
1      0.00
2      0.00
3      0.00
4      0.00
       ... 
1604   0.95
1605   0.92
1606   0.84
1612   1.00
1613   1.00
Name: degree, Length: 1089, dtype: float64